Install Needed Packages in Command Line
$ cd “C:\Users\JulieR\AppData\Local\Programs\Python\Python37-32\Scripts”
$ pip install pymongo
$ pip install vaderSentiment
$ pip install requests

Must have the MongoDB server running with processed data
    Database: "Full"
    Collection: "Cleaned"

In [ ]:
## Set up pymongo
import pymongo
from pymongo import MongoClient
client = MongoClient('localhost', 27017)
db = client.full
collection = db.cleaned

In [ ]:
## Clean line breaks (\n) from the data
for doc in collection.find({}):
    mongo_id = doc['_id']
    new_body = doc['body'].replace('\n', ' ')
    collection.update_one({'_id': mongo_id}, {"$set": {'body': new_body}}, upsert=False)

In [ ]:
## Set up Vader Sentiment
import vaderSentiment
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()

In [ ]:
## Run Vader Sentiment on all body entries
for doc in collection.find({}):
    mongo_id = doc['_id']
    all_sentiment = analyzer.polarity_scores(doc['body'])
    comp_sentiment = all_sentiment['compound']
    pos_sentiment = all_sentiment['pos']
    neu_sentiment = all_sentiment['neu'] 
    neg_sentiment = all_sentiment['neg']
    collection.update_one({'_id': mongo_id}, {"$set": {'compound': comp_sentiment, 'pos': pos_sentiment, 'neu': neu_sentiment, 'neg': neg_sentiment}}, upsert=False)

In [ ]:
## Export for Weka (no body, because the ARFF converter was having a fit with line breaks and commas)
>>> with open('CleanedCommentsForWeka.csv', 'w', newline='', encoding='utf-8') as csvfile:
    output_writer = csv.writer(csvfile, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    output_writer.writerow(["Object_ID", "Author", "Collapsed", "Collapsed_Reason", "Controversiality", "Created_UTC", "Distinguished", "Edited", "Gilded", "ID", "Is_Submitter", "Score", "Subreddit", "Pos_Score", "Neu_Score", "Neg_Score", "Compound_Score"])
    for doc in collection.find({}):
        output_writer.writerow([doc['_id'], doc['author'], doc['collapsed'], doc['collapsed_reason'], doc['controversiality'], doc['created_utc'], doc['distinguished'], doc['edited'], doc['gilded'], doc['id'], doc['is_submitter'], doc['score'], doc['subreddit'], doc['pos'], doc['neu'], doc['neg'], doc['compound']])

Export to CSV in Command Line

$ mongoexport --db full  --collection cleaned  --type=csv  --fields _id,author,body,collapsed,collapsed_reason,controversiality,created_utc,distinguished,edited,gilded,id,is_submitter,score,subreddit,compound,pos,neu,neg  --out CleanedSentiment.csv